# Build the Chatbot Framework

State-machine to handle responses using the intents model (previous) as classifier.

## Imports

In [1]:
# for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# for tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

# for files
import json

# model export / import
import pickle

## Restore data-structures

Restore the data structure previously saved.

In [2]:
data = pickle.load( open( "data/training_data.pkl", "rb" ) )

words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [3]:
with open('data/intents.json') as json_data:
    intents = json.load(json_data)

## Load Saved Model

### Model Structure

First, we define the model structure used before.

In [4]:
# reset underlying graph data
tf.reset_default_graph()

# Build Neural Network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Load Saved Model

The already trained & saved model.

In [5]:
model.load(model_file='exports/model.tflearn')

INFO:tensorflow:Restoring parameters from /notebooks/NLP/ContextualChatbot/exports/model.tflearn


## Create Bag of Words from User Input

Array with 0's and 1's fow words wich exist in the patterns (sentences) we have in the intents.

In [6]:
# list of invalid words
invalid_words = ["?"]

### Clean the sentence

In [7]:
def clean_up_sentence(sentence):
    # tokenize
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem words
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words if word not in invalid_words]
    
    return sentence_words

In [8]:
clean_up_sentence("Which mopeds do you have?")

['which', 'mop', 'do', 'you', 'hav']

### BOW function

Returns bag of words array 

0 or 1 for each word in the sentence that exists in the bag.

In [9]:
def bow(sentence, words, show_details=False):
    # tokenize
    sentence_words = clean_up_sentence(sentence)
    
    # "Tensor" of zeros
    bag = [0] * len(words)
    
    # loop through all words in the sentence tested
    for sentence_word in sentence_words:
        
        # loop through all words in the defined intents
        for index, word in enumerate(words):
            if word == sentence_word:
                bag[index] = 1
                
                if show_details:
                    print("word found in bag: %s" % word)

    return (np.array(bag))

In [10]:
bow("Are you attending today?", words, True)

word found in bag: ar
word found in bag: you
word found in bag: today


array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1])

## Create the Response Processor

### Global vars

In [11]:
context = {}
ERROR_THRESHOLD = 0.25

### Classify

First, classify the sentence based in the `patterns` and `tags` in intents.

In [12]:
def classify(sentence):
    # generate probabilities from the model
    results = model.predict( [bow(sentence, words)] )[0]
    
    # filter out predictions below threshold
    results = [[index,result] for index,result in enumerate(results) if result > ERROR_THRESHOLD]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    
    return_list = []
    
    for r in results:
        return_list.append((classes[r[0]], r[1]))
        
    # tuple of intent and probability
    return return_list

In [13]:
classify("I want a moped")

[('rental', 0.67940056)]

In [14]:
classify("Hours for today?")

[('today', 0.521593), ('opentoday', 0.31226322)]

In [15]:
classify("I only have cash for pay?")

[('payments', 0.7292963)]

### Response

Explore tags for classification results and give a random response from that tag.

In [16]:
def response(sentence, userID="123", show_details=False):
    results = classify(sentence)
    
    # if classification have results, find the matching intent tag
    if results:
        # loop all results
        while results:
            for intent in intents['intents']:
                # find the matching tag
                if intent['tag'] == results[0][0]:
                    # random response from the intent
                    return print(random.choice(intent['responses']))
            
            results.pop(0)

In [17]:
def response(sentence, userID="123", show_details=False):
    results = classify(sentence)
    
    # if classification have results, find the matching intent tag
    if results:
        # loop all results
        while results:
            for intent in intents['intents']:
                # find the matching tag
                if intent['tag'] == results[0][0]:
                    # set context for this intent
                    if 'context_set' in intent:
                        if show_details: print ('context: ', intent['context_set'])
                        
                        context[userID] = intent['context_set']

                    # check if intent is contextual an apply to user's conversation
                    if not 'context_filter' in intent \
                        or (\
                            userID in context \
                            and 'context_filter' in intent \
                            and intent['context_filter'] == context[userID]\
                            ):
                        if show_details: print('tag: ', intent['tag'] )
                        # random response from the intent
                        return print(random.choice(intent['responses']))
            
            results.pop(0)

In [18]:
response("What hours is open today?")

Our hours are 9am-9pm every day


In [19]:
response("I only have cash for pay?")

We accept most major credit cards


In [20]:
response("what cards accept?")

We accept most major credit cards


In [21]:
response("what kinds of mopeds are here")

We have Piaggio, Vespa and Yamaha mopeds


## Contextualization

In [22]:
response("can rent a moped?", show_details=True)

context:  rentalday
tag:  rental
Are you looking to rent today or later this week?


In [23]:
response("today", show_details=True)

tag:  today
For rentals today please call 1-800-MYMOPED


In [24]:
context

{'123': 'rentalday'}

In [25]:
response("hi there", show_details=True)

context:  
tag:  greeting
Hi there, how can I help?


In [26]:
response('today', show_details=True)